# Description

Link [here](https://www.kaggle.com/c/nlp-getting-started/overview)

Pretrined Bert + untrained GRU + nn.Linear

Train: 1 epoch

In [ ]:
! pip install transformers

     |████████████████████████████████| 1.9MB 5.6MB/s 
     |████████████████████████████████| 3.2MB 18.4MB/s 
     |████████████████████████████████| 890kB 34.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=b1705dc3d80f0372f5353451cc4da2a9371cd3660f8f4a53af4e407d0a32e80b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:

from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
tokens = tokenizer.tokenize('लॉन्च हो चुका है. इस स्मार्टफ़ोन में 5G कनेक्टिविटी के लिए MediaTek Dimnesty 800 U चिपसेट दिया गया है. इस वेरिएंट में कंपनी ने 120Hz रिफ़्रेश रेट वाली डिस्प्ले दी है.')

print(tokens)

['[UNK]', 'ह', '##ो', 'च', '##क', '##ा', 'ह', '.', '[UNK]', '[UNK]', 'म', '5', '##g', 'क', '##न', '##क', '##ट', '##ि', '##व', '##ि', '##ट', '##ी', 'क', 'ल', '##ि', '##ए', 'media', '##tek', 'dim', '##nes', '##ty', '800', 'u', 'च', '##ि', '##प', '##स', '##ट', 'द', '##ि', '##य', '##ा', 'ग', '##य', '##ा', 'ह', '.', '[UNK]', 'व', '##र', '##ि', '##ए', '##ट', 'म', 'क', '##प', '##न', '##ी', 'न', '120', '##h', '##z', '[UNK]', 'र', '##ट', 'व', '##ा', '##ल', '##ी', 'ड', '##ि', '##स', '##प', '##ल', 'द', '##ी', 'ह', '.']


In [ ]:
tokenizer.add_tokens(['लॉन्च'])
tokenizer.add_tokens(['आप'])

1

In [ ]:
from transformers import BertTokenizer, BertModel

bert = BertModel.from_pretrained('bert-base-uncased')

import torch.nn as nn

class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        self.bert.resize_token_embeddings(len(tokenizer)) 
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
                
        with torch.no_grad():
            embedded = self.bert(text)[0]
                
        #embedded = [batch size, sent len, emb dim]
        
        _, hidden = self.rnn(embedded)
        
        #hidden = [n layers * n directions, batch size, emb dim]
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
                
        #hidden = [batch size, hid dim]
        
        output = self.out(hidden)
        
        #output = [batch size, out dim]
        
        return output

In [ ]:
HIDDEN_DIM = 256
OUTPUT_DIM = 2
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

In [ ]:
device = 'cuda'

import torch.optim as optim

optimizer = optim.Adam(model.parameters())

# criterion = nn.BCEWithLogitsLoss()
criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [ ]:

def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

### Text dataset

In [ ]:
WORKING_DIR = '/content/drive/MyDrive/Hackerearth/Disaster Tweet'

import os
os.chdir(WORKING_DIR)


import warnings
warnings.filterwarnings("ignore")

import pandas as pd 
import numpy as np
import torch 
import torch.nn as nn
import torchtext
import matplotlib.pyplot as plt


In [ ]:
df = pd.read_csv('train.csv')
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
from torchtext.legacy import data

import re
import string 

def clean_text(text):
    text = text.lower()
    #foction de replacement
    text = re.sub(r"i'm","i am",text)
    text = re.sub(r"she's","she is",text)
    text = re.sub(r"can't","can not",text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)

    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub(r"[-()\"#/@;:<>{}=~|.?!,]", "", text)
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
class TextDataset(Dataset):
    def __init__(self, df, tok):
        self.df = df
        self.tok = tok
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, id):
        text = self.df.iloc[id]['text']
        text = self.tok.encode(text, return_tensors='pt')
        label = torch.tensor([self.df.iloc[id].get('target', 0)])
        return text, label, self.df.iloc[id]['id']


text_dataset = TextDataset(df, tokenizer)
dataloader = DataLoader(text_dataset, batch_size=1,
                        shuffle=True, num_workers=0)


In [ ]:
for batch in dataloader:
    print(batch[0].shape, batch[1].shape)
    print(batch[0])
    # [batch_size, max_length_of_sentence_in_batch]
    break

torch.Size([1, 1, 32]) torch.Size([1, 1])
tensor([[[  101,  4744,  4062, 18340,  2015,  7917, 12851,  2013,  6215,  2006,
           1001,  2845,  3675,  8299,  1024,  1013,  1013,  1056,  1012,  2522,
           1013,  1021,  2497,  2475,  2860,  2546,  2575,  4492, 24316,  1001,
           2739,   102]]])


In [ ]:
for batch in text_dataset:
    print(batch, batch[0].shape, batch[1].shape, sep='\n')
    break

(tensor([[  101,  2256, 15616,  2024,  1996,  3114,  1997,  2023,  1001,  8372,
          2089, 16455,  9641,  2149,  2035,   102]]), tensor([1]))
torch.Size([1, 16])
torch.Size([1])


In [ ]:
from tqdm import tqdm

def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in tqdm(iterator):
        
        optimizer.zero_grad()

        # print(batch)
        text_ids = batch[0].to(device)
        label = batch[1].to(device)
        
        predictions = model(text_ids)

        # print(f'Input: {text_ids.shape}, Labels: {label.shape}, Outs: {predictions.shape}\n')
        
        loss = criterion(predictions, label)
        acc = binary_accuracy(predictions, label)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
train(model, text_dataset, optimizer, criterion=criterion)

100%|██████████| 7613/7613 [05:01<00:00, 25.25it/s]


(0.49359652802127607, 0.9889662419545514)

In [ ]:
criterion

CrossEntropyLoss()

### Predict

In [ ]:
test_df = pd.read_csv('cleaned test.csv')
test_df.tail()

,id,keyword,location,text
3258,10861,NaN,NaN,earthquake safety los angeles ûò safety faste...
3259,10865,NaN,NaN,storm in ri worse than last hurricane my hard...
3260,10868,NaN,NaN,green line derailment in chicago
3261,10874,NaN,NaN,meg issues hazardous weather outlook hwo
3262,10875,NaN,NaN,cityofcalgary has activated its municipal emer...


In [ ]:
test_dataset = TextDataset(test_df, tokenizer)

In [ ]:
for batch in test_dataset:
    print(batch, '|', batch[-1].item(), '\n')
    out = model(batch[0].cuda())

    print(torch.argmax(out, dim=1).item(), out, sep='\t')
    break

(tensor([[ 101, 2074, 3047, 1037, 6659, 2482, 5823,  102]]), tensor([0]), 0) | 0 

1	tensor([[-0.5233,  1.3746]], device='cuda:0', grad_fn=<AddmmBackward>)


In [ ]:
preds = pd.DataFrame(columns=['id', 'target'])

for batch in tqdm(test_dataset):
    out = model(batch[0].cuda())
    out = torch.argmax(out, dim=1).item()

    preds = preds.append({'id': batch[-1].item(), 'target': out}, ignore_index=True)

100%|██████████| 3263/3263 [01:16<00:00, 42.87it/s]


In [110]:
preds.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [ ]:
org = set(pd.read_csv('test.csv')['id'])
org == set(preds['id']), len(org - set(preds['id'])), len(org), len(preds)

(True, 0, 3263, 3263)

In [111]:
preds.to_csv('preds.csv', index=False)

In [ ]:
# extra

set(preds['id']) - set(test_df['id'])

In [ ]:
len(set(preds['id'])), len(set(test_df['id']))

(3263, 3263)

In [ ]:
sorted(list(preds['id']), reverse=True)[:20], sorted(list(test_df['id']), reverse=True)[:20]

In [ ]:
for x in test_df['id']: 
    if x not in preds['id']:
        print(x)

In [112]:
{'Rank': 1249, 'Score': 0.75881}

{'Rank': 1249, 'Score': 0.75881}